In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import dei dati

In questa porzione di notebook avviene il caricamento del dataset da una file .csv tramite la creazione di un DataFrame Pandas. 
Un DataFrame Pandas è un "Two-dimensional, size-mutable, potentially heterogeneous tabular data." 

In [ ]:
telco_data = pd.read_csv("Telco.csv")

## Controllo del Dataset

Una volta caricato il Dataset controllo che i dati presenti all'interno siano corretti, ovvero che rispettino le seguenti caratteristiche:

1. Assenza di campi nulli o NaN 
2. Assenza di duplicati

In [ ]:
missing_data = telco_data.isnull().sum(axis=0).reset_index()

In [ ]:
missing_data

In [ ]:
NaN_data = telco_data.isna().sum(axis=0).reset_index()

In [ ]:
NaN_data

In [ ]:
duplicates = telco_data.duplicated(['customerID'], keep=False).sum(axis=0)
print(duplicates)

## Visualizzazione delle informazioni

Una volta controllata la correttezza dei dati possiamo visualizzarli utilizzando la libreria "seaborn".

Seaborn si basa su matplotlib e viene utilizzata per plottare come varia la feature di interesse Churn per ogni feature secondaria del dataset.

L'operazione è abbastanza esosa di tempo e risorse ma permette di avere una visione completa dei dati in analisi. 
Per ottenere risultati consistenti è possibile accorpare i valori "No internet service" con "No" nelle colonne d'interesse poiché significano la stessa cosa, stessa cosa per "No phone service".

In [ ]:
telco_data = telco_data.replace(to_replace=['No phone service', 'No internet service'], value='No')

Questo ciclo for permette di realizzare un grafo per ogni feature presente nel dataset, ad eccezione di Churn.

In [ ]:
for i, predictor in enumerate(telco_data.drop(columns=['Churn'])):
    plt.figure(i)
    sns.countplot(data=telco_data, x=predictor, hue='Churn')

## LabelEncoding 

Il dataset contiene valori non numerici, è necessario trasformare tutte le feature in valori numerici per renderli compatibili con gli algoritmi di classificazione

La funzione Label Encoding permette di codificare tutti i dati non numerici, i label appunto, in valori compresi tra 0 e N-1 dove N sono il numero di valori univoci presenti all'interno delle colonne

Le colonne da sistemare vengono ricavate dalla struttura dati X, che contiene l'intero dataset

La funzione lambda permette di applicare la funzione di traformazione offerta da LabelEncoder su tutte le colonne da sistemare. 

Dopo l'esecuzione dell'encoding viene stampata la tabella contenente le colonne trasformate, che potrebbe omettere alcune colonne presenti all'interno del dataset poiché non soggette ad encoding, e la tabella completa per visualizzare il contenuto di X

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = telco_data

In [ ]:
features_to_encode = X.dtypes == object

In [ ]:
columns = X.columns[features_to_encode].tolist()

In [ ]:
le = LabelEncoder()

In [ ]:
X[columns] = X[columns].apply(lambda col: le.fit_transform(col))

In [ ]:
X

## Initial Feature Selection

Dall'analisi dei dati, grazie ai grafici, è posssible eliminare alcune colonne poiché non significative per la predizione del Churn.
Le colonne che verranno eliminate, ovvero droppate, saranno: 
1. customerID poichè non è un'informazione rilevamente
2. gender poiché non presenta delle variazioni significative

In [ ]:
telco_data = telco_data.drop(columns=['customerID', 'gender'])

In [ ]:
X = telco_data

## Inizializzazione e apprendimento del modello

Il modello scelto è un classificatore RandomForest. Il classificatore RandomForest utilizza diversi classificatori Decision Tree su diversi sotto campioni del dataset di input. Ogni classificatore compierà un'operazione di fitting sul suo sottocampione. Il classificatore RandomForest utilizzerà una funzione di averaging, ovvero fa la media tra le varie predizioni, per migliorare l'accuratezza e tenere sotto controllo l'overfitting. 

Un Decision tree classifier ha il compito di predirre il valore della variabile target, nel nostro caso Churn, apprendendo delle regole di decisione a partire dalle feature del nostro dataset

Per selezionare le feature da utilizzare usiamo la funzione SelectFromModel offerta da scikit_learn. Questa funzione rimuove le features meno importanti, ovvero che incidono poco sull'accuratezza della predizione, restituendo un insieme di features che massimizzano l'accuratezza della predizione. 

SelectFromModel utilizza come metrica di valutazione un valore di threshold, che di default è la mediana tra i valori di predizione. Se il valore associato ad una feature supera la threashold verrà considerata, altrimenti verrà scartata. Il RandomForestClassifier utilizza 1000 stimatori, ovvero Decision Trees su sottocampioni di dataset, per sperimentare più combinazioni possibili, a costo di un tempo di esecuzione lievemente maggiore del solito.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [ ]:
y = telco_data.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.7)

In [ ]:
feature_selector = SelectFromModel(RandomForestClassifier(n_estimators = 1000, max_depth = 10, random_state=42))

In [ ]:
feature_selector.fit(X_train, y_train)

In [ ]:
features= feature_selector.get_support()
telco_data.columns[features]

In [ ]:
selected_features= X_train.columns[(feature_selector.get_support())]

In [ ]:
X = telco_data[selected_features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
print("Test set accuracy: {:.2f}".format(clf.score(X_test, y_test)))

In [ ]:
print("Test set predictions:", clf.predict(X_test))

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.YlGn)